In [ ]:
from google.colab import drive
import pandas as pd

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
csv_file_path = '/content/drive/MyDrive/cleaned_articles.csv'

In [ ]:
df = pd.read_csv(csv_file_path)

In [ ]:
df.head()

,title,cleaned_article
0,The symbols of Poland’s abortion protests expl...,The symbols of Polands abortion protests expla...
1,Impending Threat of Abortion Criminalization B...,Impending Threat of Abortion Criminalization B...
2,New lawsuits target state restrictions on abor...,New lawsuits target state restrictions on abor...
3,Indiana’s near-total abortion ban leads doctor...,Indianas near-total abortion ban leads doctors...
4,Legal challenge to Texas abortion law official...,Legal challenge to Texas abortion law official...


In [ ]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 7.2 MB/s eta 0:00:00


GPT 4 FOR NER

In [ ]:
import openai
import pandas as pd

# Set your OpenAI API key
openai.api_key = "sk-YduySWfF_bNwkPvjVQmae2fG-T9GrOJ7vTQ48b3VkDT3BlbkFJosMJMytz5Vh8osnwLmaOybxd23sJgJz_sTaE3P530A"

# Function to get entities using ChatGPT API
def get_entities_from_gpt(article_text):
    prompt = f"Extract the named entities from the following text:\n\n{article_text}\n\nList them along with their types (e.g., PERSON, LOCATION, ORGANIZATION, DATES)."

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    # Extract the response text
    entities_output = response['choices'][0]['message']['content']

    return entities_output

# Function to show entities and allow user selection
def show_entities_and_get_selection(article_index):
    article_text = df.loc[article_index, 'cleaned_article']

    # Get entities from GPT
    entities_output = get_entities_from_gpt(article_text)

    print("Extracted Entities from GPT:")
    print(entities_output)  # Print the output received from GPT

    # Split the output into lines and display options
    entities_lines = entities_output.strip().split('\n')

    if entities_lines:
        print("Please select the entities you want to include in the summary:")
        for i, line in enumerate(entities_lines):
            print(f"{i + 1}: {line}")

        # User selects entities
        selected_indices = input("Select entity numbers to include in the summary (comma-separated): ")

        try:
            # Sanitize and convert input to integers
            selected_indices = [int(i.strip()) for i in selected_indices.split(',')]

            # Get selected entities
            selected_entities = [entities_lines[i - 1] for i in selected_indices if 0 < i <= len(entities_lines)]
        except ValueError:
            print("Invalid input. Please enter numbers only.")
            selected_entities = []  # Reset if there was an error
        except IndexError:
            print("Some selected indices are out of range.")
            selected_entities = []  # Reset if there was an error
    else:
        selected_entities = []
        print("No entities available for selection.")

    return selected_entities

# Example usage
article_index = 0  # Example: First article
selected_entities = show_entities_and_get_selection(article_index)

print("Selected entities:", selected_entities)


LLAMA2 FOR NER

In [ ]:
!pip install huggingface_hub

In [ ]:
!pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.7 MB/s eta 0:00:00


In [25]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [26]:
import os
from huggingface_hub import login

# Retrieve the Hugging Face token from the secrets
hf_token = os.environ.get("AT_LLAMA2")

# Login to Hugging Face using the token
if hf_token:
    login(token=hf_token)

In [27]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Ensure GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the LLaMA 2 model and tokenizer with quantization config
model_name = "meta-llama/Llama-2-7b-hf"
quant_config = BitsAndBytesConfig(load_in_4bit=True)  # Replaces deprecated load_in_4bit argument
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=quant_config)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# Function to generate text using LLaMA 2
def generate_text(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)  # Ensure inputs are sent to the correct device
    with torch.no_grad():
        outputs = model.generate(inputs['input_ids'].to(device), max_length=5000)  # Adjust max_length as needed
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Function to get unique entities using LLaMA 2
def get_entities_from_llama(article_text):
    prompt = (
        f"Extract unique named entities from the following text. "
        f"These entities will be used for further summarization, so focus on identifying important keywords "
        f"that are crucial to maintaining factual accuracy in the summary. "
        f"Please categorize each entity with its type (e.g., PERSON, LOCATION, ORGANIZATION, DATES). "
        f"Here's the article:\n\n{article_text}"
    )

    response = generate_text(prompt)
    return response

# Function to show entities and allow user selection
def show_entities_and_get_selection(article_index):
    article_text = df.loc[article_index, 'cleaned_article']

    # Get entities from LLaMA 2
    entities_output = get_entities_from_llama(article_text)

    print("Extracted Entities from LLaMA 2:")
    print(entities_output)  # Print the output received from LLaMA 2

    # Split the output into lines and display options
    entities_lines = entities_output.strip().split('\n')

    if entities_lines:
        print("Please select the entities you want to include in the summary:")
        for i, line in enumerate(entities_lines):
            print(f"{i + 1}: {line}")

        # User selects entities
        selected_indices = input("Select entity numbers to include in the summary (comma-separated): ")

        try:
            # Sanitize and convert input to integers
            selected_indices = [int(i.strip()) for i in selected_indices.split(',')]

            # Get selected entities
            selected_entities = [entities_lines[i - 1] for i in selected_indices if 0 < i <= len(entities_lines)]
        except ValueError:
            print("Invalid input. Please enter numbers only.")
            selected_entities = []  # Reset if there was an error
        except IndexError:
            print("Some selected indices are out of range.")
            selected_entities = []  # Reset if there was an error
    else:
        selected_entities = []
        print("No entities available for selection.")

    return selected_entities

# Example usage
article_index = 0  # Example: First article
article_text = df.loc[article_index, 'cleaned_article']
selected_entities = show_entities_and_get_selection(article_index)

print("Selected entities:", selected_entities)

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Extracted Entities from LLaMA 2:
Extract unique named entities from the following text. These entities will be used for further summarization, so focus on identifying important keywords that are crucial to maintaining factual accuracy in the summary. Please categorize each entity with its type (e.g., PERSON, LOCATION, ORGANIZATION, DATES). Here's the article:

The symbols of Polands abortion protests explained Notes From Poland News Insights Po polsku Topics Politics Society Culture Business History Education Energy Climate Law Media Podcasts Notes from Poland Podcast The VoiCEE podcast About us Our story Editorial team Advisory Board Notes from Poland Foundation Contact Newsletters Notes from Poland Newsletter Central and Eastern Europe Newsletter Select Page The symbols of Polands abortion protests explained Oct 30, 2020 Explainers, Politics, Society By Agnieszka Wdoowska Since last weeks constitutional court ruling that introduces a near total ban on abortion in Poland, the country 